Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logistic Model
---------

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularize = tf.nn.l2_loss(weights)
  loss += 0.01*regularize

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 48.648193
Training accuracy: 10.3%
Validation accuracy: 11.8%
Loss at step 100: 12.076439
Training accuracy: 74.2%
Validation accuracy: 72.6%
Loss at step 200: 4.568440
Training accuracy: 78.9%
Validation accuracy: 77.1%
Loss at step 300: 2.005740
Training accuracy: 82.1%
Validation accuracy: 79.9%
Loss at step 400: 1.136158
Training accuracy: 83.6%
Validation accuracy: 81.5%
Loss at step 500: 0.838255
Training accuracy: 84.1%
Validation accuracy: 82.0%
Loss at step 600: 0.734952
Training accuracy: 84.3%
Validation accuracy: 82.2%
Loss at step 700: 0.698784
Training accuracy: 84.2%
Validation accuracy: 82.2%
Loss at step 800: 0.686026
Training accuracy: 84.3%
Validation accuracy: 82.3%
Test accuracy: 88.8%


NN Model
---------

In [7]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  logits = tf.matmul(hidden1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights) +
                  tf.nn.l2_loss(weights2))
  # Add the regularization term to the loss.
  loss += 0.001 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  hidden1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_logits = tf.matmul(hidden1_valid, weights2) + biases2
  valid_prediction = tf.nn.softmax(valid_logits)

  hidden1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_logits = tf.matmul(hidden1_test, weights2) + biases2
  test_prediction = tf.nn.softmax(test_logits)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 667.864929
Minibatch accuracy: 7.8%
Validation accuracy: 28.4%
Minibatch loss at step 500: 202.145645
Minibatch accuracy: 75.8%
Validation accuracy: 78.8%
Minibatch loss at step 1000: 114.008453
Minibatch accuracy: 90.6%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 69.952721
Minibatch accuracy: 81.2%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 41.397552
Minibatch accuracy: 88.3%
Validation accuracy: 85.4%
Minibatch loss at step 2500: 25.289839
Minibatch accuracy: 88.3%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 15.503264
Minibatch accuracy: 83.6%
Validation accuracy: 87.1%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  logits = tf.matmul(hidden1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights) +
                  tf.nn.l2_loss(weights2))
  # Add the regularization term to the loss.
  loss += 0.001 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  hidden1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_logits = tf.matmul(hidden1_valid, weights2) + biases2
  valid_prediction = tf.nn.softmax(valid_logits)

  hidden1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_logits = tf.matmul(hidden1_test, weights2) + biases2
  test_prediction = tf.nn.softmax(test_logits)

In [10]:
num_steps = 3001
num_batches = 3

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    if (step % ((num_steps-1)/num_batches)) == 0:
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        print('offset:',offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
offset: 0
Minibatch loss at step 0: 672.337646
Minibatch accuracy: 7.8%
Validation accuracy: 27.9%
Minibatch loss at step 500: 190.994888
Minibatch accuracy: 100.0%
Validation accuracy: 69.9%
offset: 128000
Minibatch loss at step 1000: 158.138535
Minibatch accuracy: 68.0%
Validation accuracy: 72.3%
Minibatch loss at step 1500: 70.233521
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
offset: 56128
Minibatch loss at step 2000: 62.859093
Minibatch accuracy: 73.4%
Validation accuracy: 72.8%
Minibatch loss at step 2500: 25.818987
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
offset: 184128
Minibatch loss at step 3000: 20.676083
Minibatch accuracy: 74.2%
Validation accuracy: 68.3%
Test accuracy: 74.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
 
  # implement dropout
  hidden1_drop = tf.nn.dropout(hidden1, 0.5)

  logits = tf.matmul(hidden1_drop, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  hidden1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_logits = tf.matmul(hidden1_valid, weights2) + biases2
  valid_prediction = tf.nn.softmax(valid_logits)

  hidden1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_logits = tf.matmul(hidden1_test, weights2) + biases2
  test_prediction = tf.nn.softmax(test_logits)

In [12]:
num_steps = 3001
num_batches = 3

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    if (step % ((num_steps-1)/num_batches)) == 0:
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        print('offset:',offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
offset: 0
Minibatch loss at step 0: 545.379761
Minibatch accuracy: 12.5%
Validation accuracy: 26.3%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.8%
offset: 128000
Minibatch loss at step 1000: 117.830360
Minibatch accuracy: 73.4%
Validation accuracy: 72.3%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
offset: 56128
Minibatch loss at step 2000: 202.313217
Minibatch accuracy: 75.8%
Validation accuracy: 75.9%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
offset: 184128
Minibatch loss at step 3000: 166.608551
Minibatch accuracy: 75.0%
Validation accuracy: 75.7%
Test accuracy: 82.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [72]:
batch_size = 128
hl1_nodes = 1024
hl2_nodes = 300

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hl1_nodes], stddev=0.05))
  weights2 = tf.Variable(tf.truncated_normal([hl1_nodes, hl2_nodes], stddev=0.05))
  weights3 = tf.Variable(tf.truncated_normal([hl2_nodes, num_labels], stddev=0.05))
  biases = tf.Variable(tf.zeros([hl1_nodes]))
  biases2 = tf.Variable(tf.zeros([hl2_nodes]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
 
  # implement dropout
  hidden2_drop = tf.nn.dropout(hidden2, 0.5)

  logits = tf.matmul(hidden2_drop, weights3) + biases3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step,
            200000,0.90, staircase = True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  hidden1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  hidden2_valid = tf.nn.relu(tf.matmul(hidden1_valid, weights2) + biases2)
  valid_logits = tf.matmul(hidden2_valid, weights3) + biases3
  valid_prediction = tf.nn.softmax(valid_logits)

  hidden1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  hidden2_test = tf.nn.relu(tf.matmul(hidden1_test, weights2) + biases2)
  test_logits = tf.matmul(hidden2_test, weights3) + biases3
  test_prediction = tf.nn.softmax(test_logits)

In [73]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.361415
Minibatch accuracy: 10.2%
Validation accuracy: 41.9%
Minibatch loss at step 500: 0.663648
Minibatch accuracy: 78.1%
Validation accuracy: 86.0%
Minibatch loss at step 1000: 0.403681
Minibatch accuracy: 83.6%
Validation accuracy: 87.0%
Minibatch loss at step 1500: 0.507897
Minibatch accuracy: 83.6%
Validation accuracy: 87.6%
Minibatch loss at step 2000: 0.388099
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 2500: 0.421667
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 3000: 0.395800
Minibatch accuracy: 89.8%
Validation accuracy: 88.9%
Test accuracy: 94.2%
